In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from collections import defaultdict

In [68]:
CALL:int = 0
PUT:int = 1

STOCK:int = 0
CURRENCY:int = 1

EUROPEAN:int = 0
AMERICAN:int = 1

def get_d1(S: float, X: float, T: float, sigma: float, b: float):
    d1: float = (np.log(S / X) + (b + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    return d1

def get_d2(S: float, X: float, T: float, sigma: float, b: float):
    d1 = get_d1(S, X, T, sigma, b)
    d2: float = d1 - sigma * np.sqrt(T)
    return d2

def get_d(S: float, X: float, T: float, sigma: float, b: float):
    d1 = get_d1(S, X, T, sigma, b)
    d2:float = d1 - sigma * np.sqrt(T)
    return d1, d2

def BSM_call_option(S: float, X: float, r: float, T: float, sigma: float, b: float):
    d1, d2 = get_d(S, X, T, sigma, b)
    c: float = S * np.exp((b - r) * T) * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    return c

def BSM_put_option(S: float, X: float, r: float, T: float, sigma: float, b: float):
    p = -BSM_call_option(S, X, r, T, -sigma, b)
    return p

def price_european_stock_option(S: float, X: float, r: float, 
                                T: float, sigma: float, q: float =0, option_type: int =CALL):
    b = r - q
    if option_type == CALL:
        p = BSM_call_option(S, X, r, T, sigma, b)
    elif option_type == PUT:
        p = BSM_put_option(S, X, r, T, sigma, b)
    else:
        raise Exception('ONLY CALL or PUT options sides are existed!')
    return p

def price_european_currency_option(S: float, X: float, r: float, T: float, 
                                   sigma: float, rf: float =0, option_type: int =CALL):
    p = price_european_stock_option(S, X, r, T, sigma, rf, option_type)
    return p

def CRRBinomial(S: float, X: float, r: float, T: float, 
                sigma: float, b: float, option_type: int, n: int):
    z = 1 if option_type == CALL else -1
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(b * dt) - d) / (u - d)
    Df = np.exp(-r * dt)

    option_value = []
    for i in range(n+1):
        option_value.append(max(0, z * (S * u**i * d**(n - i) - X)))
    
    return_values = [.0 for _ in range(4)]
    for j in range(n, 0, -1):
        for i in range(j):
            option_value[i] = max(z * (S * u**i * d**(j - 1 - i) - X), (p * option_value[i+1] + (1 - p) * option_value[i]) * Df)
        
        if j == 3:
            return_values[2] = ((option_value[2] - option_value[1]) / (S * u**2 - S) - \
            (option_value[1] - option_value[0]) / (S - S * d**2)) / (0.5 * (S * u**2 - S * d**2))
            return_values[3] = option_value[1]
        if j == 2:
            return_values[1] = (option_value[1] - option_value[0]) / (S * u - S * d)
            
    return_values[3] = (return_values[3] - option_value[0]) / (2 * dt) / 365
    return_values[0] = option_value[0]

    return return_values

def price_american_stock_option(S: float, X: float, r: float, T: float, sigma: float, 
                                q: float =0, option_type: int=CALL, n: int=10**2):
    b = r - q
    return CRRBinomial(S, X, r, T, sigma, b, option_type, n)

def price_american_currency_option(S: float, X: float, r: float, T: float, sigma: float, 
                                   rf: float =0, option_type:int =CALL, n: int=10**2):
    b = r - rf
    return CRRBinomial(S, X, r, T, sigma, b, option_type, n)

def price_option(S: float, X: float, r: float, T: float, sigma: float, q: float =0, 
                 option_type: int =CALL, option_class: int = EUROPEAN, asset_type:int = STOCK):
    if option_class == EUROPEAN:
        if asset_type == STOCK:
            p = price_european_stock_option(S, X, r, T, sigma, q, option_type)
        else:
            p = price_european_currency_option(S, X, r, T, sigma, q, option_type)
    else:
        if asset_type == STOCK:
            p = price_american_stock_option(S, X, r, T, sigma, q, option_type)[0]
        else:
            p = price_american_currency_option(S, X, r, T, sigma, q, option_type)[0]
    
    return p

In [1]:
import gdown

In [4]:
url = 'https://docs.google.com/spreadsheets/d/1Pj5d7imn8MNACEJabq3Adnpx6BLoV_cy/edit?usp=drive_link'
output = 'Options_call_put.xlsx'
gdown.download(url, output, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1Pj5d7imn8MNACEJabq3Adnpx6BLoV_cy
To: /home/bogdan/CMF/Options_call_put.xlsx
100%|██████████| 11.3k/11.3k [00:00<00:00, 8.47MB/s]


'Options_call_put.xlsx'

In [3]:
put_call = pd.read_excel('Options_call_put.xlsx')

In [4]:
for column in put_call.columns:
    if column != 'Strike':
        put_call[column] /= 100 # Because price of option consider 100 underlying assets (otherwise prices are too large)

In [5]:
r = 0.072
T = 70 / 365
S = 90.2
F_stock = 91.1
F_currency = 89.3

In [39]:
def implied_volatility_european_stock(S: float, X: float, r: float, T: float, q: float, 
                                      option_price: float, option_type: int =CALL, eps: float =1e-8):
    b = r - q
    implied_sigma: float = np.sqrt(np.abs(np.log(S / X) + r * T) * 2 / T)
    implied_option_price = price_european_stock_option(S, X, r, T, implied_sigma, q, option_type)
    abs_diff = np.abs(implied_option_price - option_price)
    last_abs_diff = np.inf
    while (abs_diff > eps) and (abs_diff < last_abs_diff):
        d1 = get_d1(S, X, T, implied_sigma, b)
        vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
        implied_sigma -= (implied_option_price - option_price) / vega
        implied_option_price = price_european_stock_option(S, X, r, T, implied_sigma, q, option_type)
        last_abs_diff = abs_diff
        abs_diff = np.abs(implied_option_price - option_price)

    if (abs_diff > eps) or (np.isnan(abs_diff)):
        implied_sigma = np.nan

    return implied_sigma

def implied_volatility_european_currency(S: float, X: float, r: float, T: float, rf: float, 
                                         option_price: float, option_type: int =CALL, eps: float=1e-8):
    implied_sigma = implied_volatility_european_stock(S, X, r, T, rf, option_price, option_type, eps)
    return implied_sigma

def implied_volatility_american_stock(S: float, X: float, r: float, T: float, q: float, 
                                      option_price: float, option_type: int =CALL, eps: float=1e-8, 
                                      sigma_low: float =0.005, sigma_high: float =10):
    c_low = price_american_stock_option(S, X, r, T, sigma_low, q, option_type)[0]
    c_high =  price_american_stock_option(S, X, r, T, sigma_high, q, option_type)[0]
    implied_sigma:float = sigma_low + (option_price - c_low) * (sigma_high - sigma_low) / (c_high - c_low)
    implied_option_price = price_american_stock_option(S, X, r, T, implied_sigma, q, option_type)[0]
    abs_diff = np.abs(implied_option_price - option_price)
    counter: int = 0
    while (abs_diff > eps):
        if counter >= 100:
            return np.nan
        if implied_option_price < option_price:
            sigma_low = implied_sigma
            c_low = price_american_stock_option(S, X, r, T, sigma_low, q, option_type)[0]
        else:
            sigma_high = implied_sigma
            c_high = price_american_stock_option(S, X, r, T, sigma_high, q, option_type)[0]
        
        implied_sigma = sigma_low + (option_price - c_low) * (sigma_high - sigma_low) / (c_high - c_low)
        implied_option_price = price_american_stock_option(S, X, r, T, implied_sigma, q, option_type)[0]
        abs_diff = np.abs(implied_option_price - option_price)
        counter += 1
    return implied_sigma

def implied_volatility_american_currency(S: float, X: float, r: float, T: float, rf: float, 
                                      option_price: float, option_type: int =CALL, eps: float=1e-8, 
                                      sigma_low: float =0.005, sigma_high: float =10):
    return implied_volatility_american_stock(S, X, r, T, rf, option_price, 
                                             option_type, eps, sigma_low, sigma_high)

def option_implied_volatility(S: float, X: float, r: float, T: float, q: float, option_price: float, 
                              option_type: int =CALL, option_class: int = EUROPEAN, asset_type:int = STOCK, eps: float=1e-8):
    
    if option_class == EUROPEAN:
        if asset_type == STOCK:
            implied_sigma = implied_volatility_european_stock(S, X, r, T, q, option_price, option_type, eps)
        else:
            implied_sigma = implied_volatility_european_currency(S, X, r, T, q, option_price, option_type, eps)
    else:
        if asset_type == STOCK:
            implied_sigma = implied_volatility_american_stock(S, X, r, T, q, option_price, option_type, eps)
        else:
            implied_sigma = implied_volatility_american_currency(S, X, r, T, q, option_price, option_type, eps)
    
    return implied_sigma

In [7]:
def delta_european_stock(S: float, X: float, r: float, T: float, 
                         sigma: float, q: float, option_type: int =CALL):
    b = r - q
    d1 = get_d1(S, X, T, sigma, b)
    if option_type == CALL:
        delta = np.exp(-q * T) * norm.cdf(d1)
    else:
        delta = np.exp(-q * T) * (norm.cdf(d1) - 1)
    return delta

def delta_european_currency(S: float, X: float, r: float, T: float, 
                            sigma: float, rf: float, option_type: int =CALL):
    delta = delta_european_stock(S, X, r, T, sigma, rf, option_type)
    return delta

def delta_american_stock(S: float, X: float, r: float, T: float, 
                         sigma: float, q: float, option_type: int =CALL):
    delta = price_american_stock_option(S, X, r, T, sigma, q, option_type)[1]
    return delta

def delta_american_currency(S: float, X: float, r: float, T: float, 
                            sigma: float, rf: float, option_type: int =CALL):
    delta = price_american_currency_option(S, X, r, T, sigma, rf, option_type)[1]
    return delta

def option_delta(S: float, X: float, r: float, T: float, sigma: float, q: float =0, 
                 option_type: int =CALL, option_class: int = EUROPEAN, asset_type:int = STOCK):
    if option_class == EUROPEAN:
        if asset_type == STOCK:
            delta = delta_european_stock(S, X, r, T, sigma, q, option_type)
        else:
            delta = delta_european_currency(S, X, r, T, sigma, q, option_type)
    else:
        if asset_type == STOCK:
            delta = delta_american_stock(S, X, r, T, sigma, q, option_type)
        else:
            delta = delta_american_currency(S, X, r, T, sigma, q, option_type)
    
    return delta

In [8]:
def vega_european_stock(S: float, X: float, r: float, T: float, sigma: float, q: float):
    b = r - q
    d1 = get_d1(S, X, T, sigma, b)
    vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)
    return vega

def vega_european_currency(S: float, X: float, r: float, T: float, sigma: float, rf: float):
    vega = vega_european_stock(S, X, r, T, sigma, rf)
    return vega

def vega_american_stock(S: float, X: float, r: float, T: float, sigma: float, 
                        q: float, option_type: int =CALL, dsigma: float =0.001):
    price_high = price_american_stock_option(S, X, r, T, sigma+dsigma, q, option_type)[0]
    price_low = price_american_stock_option(S, X, r, T, sigma-dsigma, q, option_type)[0]
    vega = (price_high - price_low) / (2 * dsigma)
    return vega

def vega_american_currency(S: float, X: float, r: float, T: float, sigma: float, 
                           rf: float, option_type: int =CALL, dsigma: float =0.001):
    vega = vega_american_stock(S, X, r, T, sigma, rf, option_type, dsigma)
    return vega

def option_vega(S: float, X: float, r: float, T: float, sigma: float, q: float =0, 
                 option_type: int =CALL, option_class: int = EUROPEAN, asset_type:int = STOCK):
    if option_class == EUROPEAN:
        if asset_type == STOCK:
            vega = vega_european_stock(S, X, r, T, sigma, q)
        else:
            vega = vega_european_currency(S, X, r, T, sigma, q)
    else:
        if asset_type == STOCK:
            vega = vega_american_stock(S, X, r, T, sigma, q, option_type)
        else:
            vega = vega_american_currency(S, X, r, T, sigma, q, option_type)
    
    return vega

In [9]:
def gamma_european_stock(S: float, X: float, r: float, T: float, sigma: float, q: float):
    b = r - q
    d1 = get_d1(S, X, T, sigma, b)
    gamma = norm.pdf(d1) * np.exp(-q * T) / (S * sigma * np.sqrt(T))
    return gamma

def gamma_european_currency(S: float, X: float, r: float, T: float, sigma: float, rf: float):
    gamma = gamma_european_stock(S, X, r, T, sigma, rf)
    return gamma

def gamma_american_stock(S: float, X: float, r: float, T: float, 
                         sigma: float, q: float, option_type: int =CALL):
    gamma = price_american_stock_option(S, X, r, T, sigma, q, option_type)[2]
    return gamma

def gamma_american_currency(S: float, X: float, r: float, T: float, 
                            sigma: float, rf: float, option_type: int =CALL):
    gamma = price_american_currency_option(S, X, r, T, sigma, rf, option_type)[2]
    return gamma

def option_gamma(S: float, X: float, r: float, T: float, sigma: float, q: float =0, 
                 option_type: int =CALL, option_class: int = EUROPEAN, asset_type:int = STOCK):
    if option_class == EUROPEAN:
        if asset_type == STOCK:
            gamma = gamma_european_stock(S, X, r, T, sigma, q)
        else:
            gamma = gamma_european_currency(S, X, r, T, sigma, q)
    else:
        if asset_type == STOCK:
            gamma = gamma_american_stock(S, X, r, T, sigma, q, option_type)
        else:
            gamma = gamma_american_currency(S, X, r, T, sigma, q, option_type)
    
    return gamma

In [10]:
def vanna_european_stock(S: float, X: float, r: float, T: float, sigma: float, q: float):
    b = r - q
    d1, d2 = get_d(S, X, T, sigma, b)
    vanna = -norm.pdf(d1) * d2 * np.exp(-q * T) / sigma
    return vanna

def vanna_european_currency(S: float, X: float, r: float, T: float, sigma: float, rf: float):
    vanna = vanna_european_stock(S, X, r, T, sigma, rf)
    return vanna

def vanna_american_stock(S: float, X: float, r: float, T: float, sigma: float, 
                        q: float, option_type: int =CALL, dsigma: float =0.001):
    delta_high = price_american_stock_option(S, X, r, T, sigma+dsigma, q, option_type)[1]
    delta_low = price_american_stock_option(S, X, r, T, sigma-dsigma, q, option_type)[1]
    vanna = (delta_high - delta_low) / (2 * dsigma)
    return vanna

def vanna_american_currency(S: float, X: float, r: float, T: float, sigma: float, 
                           rf: float, option_type: int =CALL, dsigma: float =0.001):
    vanna = vanna_american_stock(S, X, r, T, sigma, rf, option_type, dsigma)
    return vanna

def option_vanna(S: float, X: float, r: float, T: float, sigma: float, q: float =0, 
                 option_type: int =CALL, option_class: int = EUROPEAN, asset_type:int = STOCK):
    if option_class == EUROPEAN:
        if asset_type == STOCK:
            vanna = vanna_european_stock(S, X, r, T, sigma, q)
        else:
            vanna = vanna_european_currency(S, X, r, T, sigma, q)
    else:
        if asset_type == STOCK:
            vanna = vanna_american_stock(S, X, r, T, sigma, q, option_type)
        else:
            vanna = vanna_american_currency(S, X, r, T, sigma, q, option_type)
    
    return vanna

In [11]:
def volga_european_stock(S: float, X: float, r: float, T: float, sigma: float, q: float):
    b = r - q
    d1, d2 = get_d(S, X, T, sigma, b)
    vega = vega_european_stock(S, X, r, T, sigma, q)
    volga = vega * (d1 * d2) / sigma
    return volga

def volga_european_currency(S: float, X: float, r: float, T: float, sigma: float, rf: float):
    volga = volga_european_stock(S, X, r, T, sigma, rf)
    return volga

def volga_american_stock(S: float, X: float, r: float, T: float, sigma: float, 
                        q: float, option_type: int =CALL, dsigma: float =0.001):
    price_high = price_american_stock_option(S, X, r, T, sigma+dsigma, q, option_type)[0]
    price_mid = price_american_stock_option(S, X, r, T, sigma, q, option_type)[0]
    price_low = price_american_stock_option(S, X, r, T, sigma-dsigma, q, option_type)[0]
    volga = (price_high - 2 * price_mid + price_low) / dsigma**2
    return volga

def volga_american_currency(S: float, X: float, r: float, T: float, sigma: float, 
                            rf: float, option_type: int =CALL, dsigma: float =0.001):
    volga = volga_american_stock(S, X, r, T, sigma, rf, option_type, dsigma)
    return volga

def option_volga(S: float, X: float, r: float, T: float, sigma: float, q: float =0, 
                 option_type: int =CALL, option_class: int = EUROPEAN, asset_type:int = STOCK):
    if option_class == EUROPEAN:
        if asset_type == STOCK:
            volga = volga_european_stock(S, X, r, T, sigma, q)
        else:
            volga = volga_european_currency(S, X, r, T, sigma, q)
    else:
        if asset_type == STOCK:
            volga = volga_american_stock(S, X, r, T, sigma, q, option_type)
        else:
            volga = volga_american_currency(S, X, r, T, sigma, q, option_type)
    
    return volga

In [12]:
def theta_european_stock(S: float, X: float, r: float, T: float, 
                         sigma: float, q: float, option_type: int = CALL):
    b = r - q
    d1, d2 = get_d(S, X, T, sigma, b)
    common_theta = (-S * np.exp(-q * T) * norm.pdf(d1) * sigma) / (2 * np.sqrt(T))
    if option_type == CALL:
        drift = -q * S * np.exp(-q * T) * norm.cdf(d1) + r * X * np.exp(-r * T) * norm.cdf(d2)
    else:
        drift = q * S * np.exp(-q * T) * norm.cdf(-d1) - r * X * np.exp(-r * T) * norm.cdf(-d2)
    theta = common_theta - drift

    return theta

def theta_european_currency(S: float, X: float, r: float, T: float, 
                            sigma: float, rf: float, option_type: int = CALL):
    theta = theta_european_stock(S, X, r, T, sigma, rf, option_type)
    return theta

def theta_american_stock(S: float, X: float, r: float, T: float, sigma: float, 
                         q: float, option_type: int = CALL):
    theta = price_american_stock_option(S, X, r, T, sigma, q, option_type)[3]
    return theta

def theta_american_currency(S: float, X: float, r: float, T: float, 
                            sigma: float, rf: float, option_type: int = CALL):
    theta = price_american_currency_option(S, X, r, T, sigma, rf, option_type)[3]
    return theta

def option_theta(S: float, X: float, r: float, T: float, sigma: float, q: float =0, 
                 option_type: int =CALL, option_class: int = EUROPEAN, asset_type:int = STOCK):
    if option_class == EUROPEAN:
        if asset_type == STOCK:
            theta = theta_european_stock(S, X, r, T, sigma, q, option_type)
        else:
            theta = theta_european_currency(S, X, r, T, sigma, q, option_type)
    else:
        if asset_type == STOCK:
            theta = theta_american_stock(S, X, r, T, sigma, q, option_type)
        else:
            theta = theta_american_currency(S, X, r, T, sigma, q, option_type)
    
    return theta

In [13]:
q = 1 + r * T - F_stock / S # dividends
rf = 1 + r * T - F_currency / S  # risk free rate for foreign currency

In [40]:
def get_delta_vega_gamma_vanna_volga_theta(S, X, r, T, sigma, q, option_type, option_class, asset_type):
    delta = option_delta(S, X, r, T, sigma, q, option_type, option_class, asset_type)
    vega = option_vega(S, X, r, T, sigma, q, option_type, option_class, asset_type)
    gamma = option_gamma(S, X, r, T, sigma, q, option_type, option_class, asset_type)
    vanna = option_vanna(S, X, r, T, sigma, q, option_type, option_class, asset_type)
    volga = option_volga(S, X, r, T, sigma, q, option_type, option_class, asset_type)
    theta = option_theta(S, X, r, T, sigma, q, option_type, option_class, asset_type)
    return [delta, vega, gamma, vanna, volga, theta]

def calculate_parameters(data_file: pd.DataFrame, option_class: int = EUROPEAN, 
                         asset_type:int = STOCK, q:float = q):
    calculated_parameters = defaultdict(dict)

    for _, row in data_file.iterrows():
        X = row['Strike']
        if ~np.isnan(row['Call: bid']):
            option_price = row['Call: bid']
            implied_volatility = option_implied_volatility(S, X, r, T, q, option_price, 
                                                           CALL, option_class, asset_type)

            calculated_parameters[X]['call_bid'] = [implied_volatility]
            calculated_parameters[X]['call_bid'] += get_delta_vega_gamma_vanna_volga_theta(S, X, r, T, implied_volatility, 
                                                                                           q, CALL, option_class, asset_type)
        if ~np.isnan(row['Call: offer']):
            option_price = row['Call: offer']
            implied_volatility = option_implied_volatility(S, X, r, T, q, option_price, 
                                                           CALL, option_class, asset_type)
    
            calculated_parameters[X]['call_offer'] = [implied_volatility]
            calculated_parameters[X]['call_offer'] += get_delta_vega_gamma_vanna_volga_theta(S, X, r, T, implied_volatility, 
                                                                                          q, CALL, option_class, asset_type)
        if ~np.isnan(row['Put: bid']):
            option_price = row['Put: bid']
            implied_volatility = option_implied_volatility(S, X, r, T, q, option_price, 
                                                           PUT, option_class, asset_type)
                                                           
            calculated_parameters[X]['put_bid'] = [implied_volatility]
            calculated_parameters[X]['put_bid'] += get_delta_vega_gamma_vanna_volga_theta(S, X, r, T, implied_volatility, 
                                                                                          q, PUT, option_class, asset_type)
        if ~np.isnan(row['Put: offer']):
            option_price = row['Put: offer']
            implied_volatility = option_implied_volatility(S, X, r, T, q, option_price, 
                                                           PUT, option_class, asset_type)
            calculated_parameters[X]['put_offer'] = [implied_volatility]
            calculated_parameters[X]['put_offer'] += get_delta_vega_gamma_vanna_volga_theta(S, X, r, T, implied_volatility, 
                                                                                          q, PUT, option_class, asset_type)

    return calculated_parameters

returning arrays are have these structure:
[implied_volatility, delta, vega, gamma, vonna, volga, theta]

In [41]:
calculated_parameters_european_stock = calculate_parameters(put_call, EUROPEAN, STOCK, q=q)

/tmp/ipykernel_973/4111660565.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  implied_sigma -= (implied_option_price - option_price) / vega
/tmp/ipykernel_973/866554982.py:11: RuntimeWarning: invalid value encountered in scalar divide
  d1: float = (np.log(S / X) + (b + sigma**2 / 2) * T) / (sigma * np.sqrt(T))


In [56]:
calculated_parameters_european_stock[70]['call_bid']

[3.825358010729925,
 0.8399428165019652,
 9.582012567637438,
 0.0016053376783551718,
 0.0430234509560195,
 -1.6939710883780477,
 -96.51009619879837]

In [42]:
calculated_parameters_european_currency = calculate_parameters(put_call, EUROPEAN, CURRENCY, q=rf)

/tmp/ipykernel_973/4111660565.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  implied_sigma -= (implied_option_price - option_price) / vega
/tmp/ipykernel_973/866554982.py:11: RuntimeWarning: invalid value encountered in scalar divide
  d1: float = (np.log(S / X) + (b + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
/tmp/ipykernel_973/866554982.py:11: RuntimeWarning: overflow encountered in scalar power
  d1: float = (np.log(S / X) + (b + sigma**2 / 2) * T) / (sigma * np.sqrt(T))


In [64]:
calculated_parameters_european_currency[89.5]['call_offer']

[1.8212771454490264,
 0.6597972308184797,
 14.361672041510735,
 0.005053714651249316,
 0.07534590865842819,
 -1.250484739401959,
 -69.02142517889828]

In [17]:
calculated_parameters_american_stock = calculate_parameters(put_call, AMERICAN, STOCK, q)

/tmp/ipykernel_973/866554982.py:60: RuntimeWarning: overflow encountered in scalar power
  option_value.append(max(0, z * (S * u**i * d**(n - i) - X)))
/tmp/ipykernel_973/866554982.py:60: RuntimeWarning: invalid value encountered in scalar multiply
  option_value.append(max(0, z * (S * u**i * d**(n - i) - X)))
/tmp/ipykernel_973/866554982.py:65: RuntimeWarning: overflow encountered in scalar power
  option_value[i] = max(z * (S * u**i * d**(j - 1 - i) - X), (p * option_value[i+1] + (1 - p) * option_value[i]) * Df)
/tmp/ipykernel_973/866554982.py:65: RuntimeWarning: invalid value encountered in scalar multiply
  option_value[i] = max(z * (S * u**i * d**(j - 1 - i) - X), (p * option_value[i+1] + (1 - p) * option_value[i]) * Df)
/tmp/ipykernel_973/866554982.py:68: RuntimeWarning: invalid value encountered in scalar subtract
  return_values[2] = ((option_value[2] - option_value[1]) / (S * u**2 - S) - \
/tmp/ipykernel_973/866554982.py:69: RuntimeWarning: invalid value encountered in scalar 

In [60]:
calculated_parameters_american_stock[69]['put_bid']

[0.41299738430330035,
 -0.050423567280395355,
 3.981242614288921,
 0.006399013131485631,
 -0.35722473475376104,
 2.028797334630017,
 -0.0112437211012123]

In [18]:
calculated_parameters_american_currency = calculate_parameters(put_call, AMERICAN, CURRENCY, rf)

/tmp/ipykernel_973/866554982.py:60: RuntimeWarning: overflow encountered in scalar power
  option_value.append(max(0, z * (S * u**i * d**(n - i) - X)))
/tmp/ipykernel_973/866554982.py:60: RuntimeWarning: invalid value encountered in scalar multiply
  option_value.append(max(0, z * (S * u**i * d**(n - i) - X)))
/tmp/ipykernel_973/866554982.py:65: RuntimeWarning: overflow encountered in scalar power
  option_value[i] = max(z * (S * u**i * d**(j - 1 - i) - X), (p * option_value[i+1] + (1 - p) * option_value[i]) * Df)
/tmp/ipykernel_973/866554982.py:65: RuntimeWarning: invalid value encountered in scalar multiply
  option_value[i] = max(z * (S * u**i * d**(j - 1 - i) - X), (p * option_value[i+1] + (1 - p) * option_value[i]) * Df)
/tmp/ipykernel_973/866554982.py:68: RuntimeWarning: invalid value encountered in scalar subtract
  return_values[2] = ((option_value[2] - option_value[1]) / (S * u**2 - S) - \
/tmp/ipykernel_973/866554982.py:69: RuntimeWarning: invalid value encountered in scalar 

In [63]:
calculated_parameters_american_currency[75]['put_offer']

[0.5197704102546188,
 -0.1671830110031023,
 9.249231249418788,
 0.012319306902470933,
 -0.33381774469976033,
 3.793500366278124,
 -0.0347115323291041]

In [69]:
def calculate_option_prices(parameters, data_file: pd.DataFrame, option_class: int = EUROPEAN, 
                         asset_type:int = STOCK, q:float = q):
    calculated_prices = defaultdict(dict)

    for _, row in data_file.iterrows():
        X = row['Strike']
        if np.isnan(row['Call: bid']):
            sigma = parameters[X]['put_bid'][0] 
            if np.isnan(sigma): sigma = parameters[X]['put_offer'][0]
            if ~np.isnan(sigma):
                price = price_option(S, X, r, T, sigma, q, CALL, option_class, asset_type)
                calculated_prices[X]['call_bid'] = price
        if np.isnan(row['Call: offer']):
            sigma = parameters[X]['put_offer'][0] 
            if np.isnan(sigma): sigma = parameters[X]['put_bid'][0]
            if ~np.isnan(sigma):
                price = price_option(S, X, r, T, sigma, q, CALL, option_class, asset_type)
                calculated_prices[X]['call_offer'] = price
        if np.isnan(row['Put: bid']):
            sigma = parameters[X]['call_bid'][0] 
            if np.isnan(sigma): sigma = parameters[X]['call_offer'][0]
            if ~np.isnan(sigma):
                price = price_option(S, X, r, T, sigma, q, PUT, option_class, asset_type)
                calculated_prices[X]['put_bid'] = price
        if np.isnan(row['Put: offer']):
            sigma = parameters[X]['call_offer'][0] 
            if np.isnan(sigma): sigma = parameters[X]['call_bid'][0]
            if ~np.isnan(sigma):
                price = price_option(S, X, r, T, sigma, q, PUT, option_class, asset_type)
                calculated_prices[X]['put_offer'] = price

    return calculated_prices

In [70]:
calculated_prices_european_stock = calculate_option_prices(calculated_parameters_european_stock, put_call, EUROPEAN, STOCK, q)
calculated_prices_european_currency = calculate_option_prices(calculated_parameters_european_currency, put_call, EUROPEAN, CURRENCY, rf)
calculated_prices_american_stock = calculate_option_prices(calculated_parameters_american_stock, put_call, AMERICAN, STOCK, q)
calculated_prices_american_currency = calculate_option_prices(calculated_parameters_american_currency, put_call, AMERICAN, CURRENCY, rf)

In [71]:
calculated_prices_european_stock[65]

{'call_bid': 26.42512978998119, 'call_offer': 27.155129789981196}

In [72]:
calculated_prices_european_currency[74.5]

{'call_bid': 17.28111361605903, 'call_offer': 18.791113616059697}

In [73]:
calculated_prices_american_stock[93.5]

{'put_offer': 17.711932179995056}

In [74]:
calculated_prices_american_currency[100]

{'put_offer': 13.761257726764663}